In [1]:
using Pkg
Pkg.add("CSV")
    

In [14]:
using CSV, DataFrames

In [15]:
station_list = CSV.read("dat/_station_list.csv", DataFrame)

PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]

filter!(row -> row.Province ∈ PROVINCES, station_list) #on choisit les station dans la zone etudiee


,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


In [16]:
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `C:\Users\leogu\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\leogu\.julia\environments\v1.6\Manifest.toml`


In [17]:
using Distributions

In [18]:
Gumbel = GeneralizedExtremeValue(0, 1, 0)

GeneralizedExtremeValue{Float64}(μ=0.0, σ=1.0, ξ=0.0)

In [21]:
loglikelihood(Gumbel, [1, 0, 0.5])    

-3.474410100884076

In [2]:
using Extremes, Distributions, GMRF

    Updating registry at `C:\Users\leogu\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `C:\Users\leogu\.julia\environments\v1.6\Project.toml`
  [429524aa] + Optim v0.22.0
  No Changes to `C:\Users\leogu\.julia\environments\v1.6\Manifest.toml`


In [3]:
function f(x)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

f (generic function with 1 method)

In [36]:
using Optim, NLSolversBase, Distributions, StatsBase, Extremes, Statistics

In [39]:
function logL(Y::Vector{Float64},μ::Real,σ::Real,ξ::Real)
    G = GeneralizedExtremeValue(μ, σ, ξ)
    return loglikelihood(G, Y)
end

logL (generic function with 2 methods)

In [57]:
function GEVparameters(Y::Vector{Float64})
    function f(p::Vector{Float64})
        return -logL(Y, p[1], p[2], p[3])
    end
    
    μ₀ = mean(Y)
    σ₀ = std(Y)
    ξ₀ = 0
    p₀ = [μ₀, σ₀, ξ₀]
    
    res = optimize(f, p₀)
    
    if Optim.converged(res)
         p = Optim.minimizer(res)
    else
        @warn "The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
        p = p₀
    end
    
    return p
end

GEVparameters (generic function with 2 methods)

In [58]:
GEVparameters([43, 54, 66, 32, 41.6])

3-element Vector{Float64}:
 42.22565849775067
 10.017784250169967
 -0.09163450812761938

In [2]:
using Extremes, Dates, DataFrames, Distributions, Gadfly

LoadError: ArgumentError: Package Gadfly not found in current path:
- Run `import Pkg; Pkg.add("Gadfly")` to install the Gadfly package.
